## Download de Bibliotecas Necessárias

In [ ]:
!pip3 install datasets==2.21.0 transformers sacrebleu torch sentencepiece transformers[sentencepiece]

In [ ]:
from datasets import load_dataset, load_metric, concatenate_datasets

In [ ]:
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM,
                          DataCollatorForSeq2Seq, Seq2SeqTrainingArguments,
                          Seq2SeqTrainer)

## 1. Importação dos Datasets a Serem Utilizados

### 1.1 FAPESP

In [ ]:
fapesp = load_dataset('caiogomide/dataset_fapesp')

In [ ]:
fapesp = fapesp.rename_column('textos_pt','texto_portugues')

### 1.2 Documentações Técnicas

In [ ]:
documentacoes_tecnicas = load_dataset('caiogomide/dataset_documentacoes_tecnicas')

### 1.3 Mesclagem dos Datasets

In [ ]:
dataset_treinamento = concatenate_datasets([fapesp['train'], documentacoes_tecnicas['train']]).shuffle(seed=42)

In [ ]:
dataset_eval = concatenate_datasets([fapesp['validation'], documentacoes_tecnicas['validation']]).shuffle(seed=42)

## 2. Fine-tuning do modelo escolhido

In [ ]:
# Seleção do modelo
model_checkpoint = 'Helsinki-NLP/opus-mt-tc-big-en-pt'

In [ ]:
# Seleção de métrica
metric = load_metric('sacrebleu')

### 2.1 Pré-processamento de Dados

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
prefix = ""
max_input_length = 256
max_target_length = 256
source_lang = 'texto_ingles'
target_lang = 'texto_portugues'

In [ ]:
def preprocessamento(exemplos):
  # Textos em inglês
  inputs = [prefix + ex for ex in exemplos['texto_ingles']]
  # Correspondentes em portugues
  targets = [ex for ex in exemplos['texto_portugues']]
  # Tokeniza o input
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
  # Tokeniza o output
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [ ]:
tokenized_dataset_treinamento = dataset_treinamento.map(preprocessamento, batched=True)

In [ ]:
tokenized_dataset_eval = dataset_eval.map(preprocessamento, batched=True)

### 2.3 Definição de parâmetros de fine-tuning

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
# Definições de parâmetros para treinamento do modelo
batch_size = 12
model_name = model_checkpoint.split('/')[-1]

args = Seq2SeqTrainingArguments(
    f'{model_name}-finetuned-final',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=6,
    predict_with_generate=True,
    load_best_model_at_end=True,
    save_strategy = "epoch"
  )

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

### 2.4 Fine-tuning final

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset_treinamento,
    eval_dataset=tokenized_dataset_eval,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()